# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

In [33]:
import pandas as pd
import numpy as np
from math import log
import pprint

# Load house sales data (train and test databases)

In [3]:
# dnames_list = ["id","date","price","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode","lat","long","sqft_living15","sqft_lot15"]
# Data types
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 
              'sqft_lot':int, 'view':int}

train_datafile = 'kc_house_train_data.csv'
train_data = pd.read_csv(train_datafile, dtype=dtype_dict)

test_datafile = 'kc_house_test_data.csv'
test_data = pd.read_csv(test_datafile, dtype=dtype_dict)

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

In [18]:
for data in [train_data, test_data]:
    # Using .apply:
    # - Apply will run through axis=0 of the dataframe (or dataseries), 
    #   passing each to the function
    #   - Default for DF is passing columns.  axis=1 passes rows.
    # - If passing a dataframe with axis=1, can access the cols of that DF within a lambda
    #   function just like normal 
    # - Can also define your own func instead of a lambda, ie:
    #   def somefunc(x):  # where x is expected to be the row or col
    #     return somethingElse
    
    # Squaring makes larger values more important
    data['bedrooms_squared'] = data['bedrooms'].apply(lambda x: x**2)
    # Interaction variable (sees combination of both)
    data['bed_bath_rooms'] = data.apply(lambda x: x['bedrooms'] * x['bathrooms'], axis=1)
    # Taking log brings larger values closer together and spreads out smaller ones
    data['log_sqft_living'] = data['sqft_living'].apply(lambda x: log(x))
    # This is not a useful variable, but will be an example later
    data['lat_plus_long'] = data.apply(lambda x: x['lat'] * x['long'], axis=1)
    
print("A sample of the new dataframes")
print("Train data:")
print(train_data.head())
print("Test data:")
print(test_data.head())


A sample of the new dataframes
Train data:
           id             date   price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900         3       1.00         1180   
1  6414100192  20141209T000000  538000         3       2.25         2570   
2  5631500400  20150225T000000  180000         2       1.00          770   
3  2487200875  20141209T000000  604000         4       3.00         1960   
4  1954400510  20150218T000000  510000         3       2.00         1680   

   sqft_lot floors  waterfront  view      ...        zipcode      lat  \
0      5650      1           0     0      ...          98178  47.5112   
1      7242      2           0     0      ...          98125  47.7210   
2     10000      1           0     0      ...          98028  47.7379   
3      5000      1           0     0      ...          98136  47.5208   
4      8080      1           0     0      ...          98074  47.6168   

      long  sqft_living15  sqft_lot15  bedrooms_squared bedro

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [25]:
for field in ["bedrooms_squared", "bed_bath_rooms", "log_sqft_living", "lat_plus_long"]:
    print("Mean of {} for TEST data is: {:.2f}".format(field,test_data[field].mean()))


Mean of bedrooms_squared for TEST data is: 12.45
Mean of bed_bath_rooms for TEST data is: 7.50
Mean of log_sqft_living for TEST data is: 7.55
Mean of lat_plus_long for TEST data is: -5812.99


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

**Unlike the course example with graphlab, I will use sklearn**

In [58]:
from sklearn import linear_model
# Notes on use: http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html

Define the features for each model

In [42]:
features = {}
# x1 included as a check.  This is the same as assignment 1 which we did analytically.  
# Should give results: intercept~=-47116.08, slope~=281.96
features['x1'] = ['sqft_living'] # Included as a check - this is from assignment 1 and coef should be ~281.95
features['m1'] = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
features['m2'] = features['m1'] + ['bed_bath_rooms']
features['m3'] = features['m2'] + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
pprint.pprint(features)

{'m1': ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long'],
 'm2': ['sqft_living',
        'bedrooms',
        'bathrooms',
        'lat',
        'long',
        'bed_bath_rooms'],
 'm3': ['sqft_living',
        'bedrooms',
        'bathrooms',
        'lat',
        'long',
        'bed_bath_rooms',
        'bedrooms_squared',
        'log_sqft_living',
        'lat_plus_long'],
 'x1': ['sqft_living']}


Create the linear regression object

In [43]:
regr = {}
for m in features.keys():
    regr[m] = linear_model.LinearRegression()

Train the model using the training dataset

In [52]:
for m in sorted(features.keys()):
    x = train_data[features[m]]
    y = train_data['price']
    regr[m].fit(x, y)
    print("Model {}:".format(m))
    print("\tintercept={:.2f}".format(regr[m].intercept_))
    print("\tcoef(s)={}".format(regr[m].coef_))


Model m1:
	intercept=-69075726.79
	coef(s)=[  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
  -3.09374351e+05]
Model m2:
	intercept=-66867968.87
	coef(s)=[  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
  -2.94298969e+05   2.55796520e+04]
Model m3:
	intercept=6610624555.28
	coef(s)=[  5.31964492e+02   3.66338204e+04   6.75006294e+04  -1.39665060e+08
   5.43198511e+07  -9.02007090e+03  -6.96138493e+03  -5.61309405e+05
  -1.14822353e+06]
Model x1:
	intercept=-47116.08
	coef(s)=[ 281.95883963]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

In [56]:
for m in ['1', '2']:
    print("Coefficient for bathrooms in model {} = {}".format(m, regr['m'+m].coef_[2]))

Coefficient for bathrooms in model 1 = 15706.74208273467
Coefficient for bathrooms in model 2 = -71461.30829275941


For model 1, an increase in the number of bathrooms leads to an increase in the sale price of a house.  However, in model 2, the number of bathrooms strongly decreases the sale price of a house.  The difference here is the context of the model.  For model 2, the interaction between bedrooms and bathrooms is accounted for in a separate feature, thus changing the influence of the pure number of bathrooms.  Although number of bathrooms (and bedrooms) here is a negative driver, it would also be a positive driver in the interaction feature.  

# Making predictions

In [76]:
# Sample of prediction using model
thisdata = train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']]
regr['m1'].predict(thisdata)

array([ 244657.18811044,  855689.66538486,  318101.67899464, ...,
        528928.42823836,  356549.38348044,  317948.91207275])

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [103]:
def get_RSS(model, data, outcome):
#     print("outcome={}".format(outcome))

    # First get the predictions
    pred = model.predict(data)
#     print("pred   ={}".format(pred))

    # Then compute the residuals/errors
    res = pred - outcome
#     print("res    ={}".format(res))
    
    # Then square and add them up
    RSS = (res * res).sum()
#     print("RSS    ={}".format(RSS))
    
    return RSS

In [102]:
rows = range(0,4)
# I'm a little sloppy here - I could also change these to np.arrays first, but they work as DF's as well
get_RSS(regr['m1'], train_data[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']].iloc[rows], train_data['price'].iloc[rows])

outcome=0    221900
1    538000
2    180000
3    604000
Name: price, dtype: float64
pred   =[ 244657.18811044  855689.66538486  318101.67899464  508443.53992161]
res    =0     22757.188110
1    317689.665385
2    138101.678995
3    -95556.460078
Name: price, dtype: float64
RSS    =129647723906.889


129647723906.889

# Comparing multiple models
Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.
**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

In [104]:
output = 'price'
for m in ['m1', 'm2', 'm3']:
    RSS = get_RSS(regr[m], train_data[features[m]], train_data[output])
    print("RSS for model {}: {}".format(m, RSS))

RSS for model m1: 967879963049549.6
RSS for model m2: 958419635074068.1
RSS for model m3: 895927073789814.2


Model 3 has the lowest RSS on the training data

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [105]:
output = 'price'
for m in ['m1', 'm2', 'm3']:
    RSS = get_RSS(regr[m], test_data[features[m]], test_data[output])
    print("RSS for model {}: {}".format(m, RSS))

RSS for model m1: 225500469795489.9
RSS for model m2: 223377462976466.47
RSS for model m3: 258804831454246.56


Model 2(!!) has the lowest RSS on the TEST data (different from the training data!).  This is because model 3, although it is well trained to its dataset (hence the low training RSS), includes a nonsense parameter of lat+long which actually hurts the prediction of house prices on the test data. 